In [ ]:
###################instalando as bibliotecas
%pip install pandas 
%pip install requests
%pip install shutil
%pip install pandas sqlalchemy psycopg2

In [1]:
###################importando as bibliotecas 
import pandas as pd
import os
import requests
import shutil
from sqlalchemy import create_engine
from sqlalchemy import create_engine, text



In [2]:
# 🔐 Conexão com o banco PostgreSQL (fiosaude)
usuario = 'postgres'
senha = '7560221'
host = 'localhost'  # ou IP do servidor
porta = '5432'
banco = 'bases_cm'

# Cria engine de conexão
engine = create_engine(f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}')

# 🧠 Função para executar comandos SQL
def executar_sql(comando_sql):
    with engine.connect() as conexao:
        conexao.execute(text(comando_sql))
        conexao.commit()


In [2]:
# 🔐 Conexão com o banco PostgreSQL (casa)
usuario = 'postgres'
senha = '7560221'
host = 'localhost'  # ou IP do servidor
porta = '5432'
banco = 'sis_datasus'

# Cria engine de conexão
engine = create_engine(f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}')

# 🧠 Função para executar comandos SQL
def executar_sql(comando_sql):
    with engine.connect() as conexao:
        conexao.execute(text(comando_sql))
        conexao.commit()

***01. PARAMETROS***

In [3]:
#############################PARAMETROS


############### ANOMES
anomes = (
'202301','202302','202303','202304','202305','202306','202307','202308','202309','202310','202311','202312',
'202401','202402','202403','202404','202405','202406','202407','202408','202409','202410','202411','202412'
)
anomes = pd.DataFrame (anomes)

############### UF
ano = (
"2023",
"2024" 
)
ano = pd.DataFrame (ano)


############### UF
uf = (
"RO",
"AC",
"AM",
"RR",
"PA",
"AP",
"TO",
"MA",
"PI",
"CE",
"RN",
"PB",
"PE",
"AL",
"SE",
"BA",
"ES",
"PR",
"SC",
"RS",
"MS",
"MT",
"GO",
"DF",
"MG",
"RJ"
#"SP"     
)
uf = pd.DataFrame (uf)

***02. AMBULATORIAL***

*2.1 CONSOLIDADO*

In [ ]:
# Caminho base dos arquivos
#caminho_base = 'C:/Users/luis.lobato/OneDrive - fiosaude.org.br/01. BACKUP/02. EVENTOS FIOCRUZ/DTISS/AMBULATORIAL/'
caminho_base = 'F:/DTISS/AMBULATORIAL/'
# Parâmetros
n = anomes.shape[0]
m = uf.shape[0]

# 🔁 Loop para abrir e carregar os arquivos
for i in range(0,n):      #anomes
    for j in range(0,1):  #uf
        uf_atual = uf.iloc[j, 0]
        anomes_atual = anomes.iloc[i, 0]
        caminho_arquivo = os.path.join(caminho_base, uf_atual, f'{uf_atual}_{anomes_atual}_AMB_CONS.csv')
        
        try:
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='utf-8')
            df.columns = [
                "id_evento_atencao_saude",
                "id_plano",
                "faixa_etaria",
                "sexo",
                "cd_municipio_beneficiario",
                "porte",
                "cd_modalidade",
                "nm_modalidade",
                "cd_municipio_prufr",
                "uf_prufr",
                "dt_realizacao",
                "cd_carater_atendimento",
                "tipo_atendimento",
                "regime_atendimento",
                "cd_motivo_saida",
                "cbo",
                "tipo_consulta",
                "saude_ocupacional",
                "ind_acidente_doenca",
                "lg_valor_preestabelecido"
            ]
            
            # 🧭 Define nome da tabela
            nome_tabela = f'{uf_atual.lower()}_{anomes_atual}_amb_cons'
            
            # 🚀 Carrega no PostgreSQL
            df.to_sql(nome_tabela, engine, schema='dtiss2', if_exists='replace', index=False)
            print(f'Tabela {nome_tabela} carregada com sucesso.')
        
        except FileNotFoundError:
            print(f'Arquivo {uf_atual}_{anomes_atual} não encontrado.')
        except Exception as e:
            print(f'Erro ao carregar {uf_atual}_{anomes_atual}: {e}')



*2.2 DETALHADO*

In [ ]:
# Caminho base dos arquivos
#caminho_base = 'C:/Users/luis.lobato/OneDrive - fiosaude.org.br/01. BACKUP/02. EVENTOS FIOCRUZ/DTISS/AMBULATORIAL/'
caminho_base = 'F:/DTISS/AMBULATORIAL/'
# Parâmetros
n = anomes.shape[0]
m = uf.shape[0]

# 🔁 Loop para abrir e carregar os arquivos
for i in range(0,n):      #anomes
    for j in range(0,m):  #uf
        uf_atual = uf.iloc[j,0]
        anomes_atual = anomes.iloc[i, 0]
        caminho_arquivo = os.path.join(caminho_base, uf_atual, f'{uf_atual}_{anomes_atual}_AMB_DET.csv')
        
        try:
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='utf-8')
            df.columns = [
    "id_evento_atencao_saude",
    "uf_prufr",
    "dt_realizacao",
    "cd_procedimento",
    "cd_tabela_referencia",
    "qt_item_evento_informado",
    "vl_item_evento_informado",
    "vl_item_pago_fornecedor",
    "unidade_medida",
    "ind_pacote",
    "ind_tabela_propria"
            ]
            
            # 🧭 Define nome da tabela
            nome_tabela = f'{uf_atual.lower()}_{anomes_atual}_amb_det'
            
            # 🚀 Carrega no PostgreSQL
            df.to_sql(nome_tabela, engine, schema='dtiss2', if_exists='replace', index=False)
            print(f'Tabela {nome_tabela} carregada com sucesso.')
        
        except FileNotFoundError:
            print(f'Arquivo {uf_atual}_{anomes_atual} não encontrado.')
        except Exception as e:
            print(f'Erro ao carregar {uf_atual}_{anomes_atual}: {e}')

*2.3 FORMATANDO VARIAVEIS DO DETALHADO*

In [ ]:
# Parâmetros
n = anomes.shape[0]
m = uf.shape[0]

for i in range(1,n):      # anomes
    for j in range(0,1):  # uf
        uf_atual = uf.iloc[j, 0]
        anomes_atual = anomes.iloc[i, 0]

        # DET - Transformações e criação de ano
        s1 = f'CREATE TABLE dtiss2.{uf_atual}_{anomes_atual}_amb_det_1 AS '
        s2 = '''
            SELECT
                id_evento_atencao_saude,
                uf_prestador,
                dt_realizacao,
                cd_procedimento,
                cd_tabela_referencia,
                SUM(COALESCE(NULLIF(REPLACE(qt_item_evento_informado, ',', '.'), '')::DECIMAL(1000,2), 0)) AS qt_procedimento,
                SUM(COALESCE(NULLIF(REPLACE(vl_item_evento_informado, ',', '.'), '')::DECIMAL(1000,2), 0)) AS vl_procedimento,
                SUM(COALESCE(NULLIF(REPLACE(vl_item_pago_fornecedor, ',', '.'), '')::DECIMAL(1000,2), 0)) AS vl_procedimento_fornecedor,
                ind_pacote,
                ind_tabela_propria,
                NULL AS lg_outlier,
                SUBSTRING(dt_realizacao, 1, 4) AS ano_realizacao
        '''
        s3 = f' FROM dtiss2.{uf_atual}_{anomes_atual}_amb_det '
        s4 = '''
            GROUP BY
                id_evento_atencao_saude,
                uf_prestador,
                dt_realizacao,
                cd_procedimento,
                cd_tabela_referencia,
                ind_pacote,
                ind_tabela_propria,
                lg_outlier,
                SUBSTRING(dt_realizacao, 1, 4)
        '''
        executar_sql(s1 + s2 + s3 + s4)

        # Drop tabela antiga
        executar_sql(f'DROP TABLE dtiss2.{uf_atual}_{anomes_atual}_amb_det')

        # Renomear nova tabela
        executar_sql(f'ALTER TABLE dtiss2.{uf_atual}_{anomes_atual}_amb_det_1 RENAME TO {uf_atual}_{anomes_atual}_amb_det')

        # CONS - Transformações e criação de ano
        s1 = f'CREATE TABLE dtiss2.{uf_atual}_{anomes_atual}_amb_cons_1 AS '
        s2 = '''
            SELECT
                id_evento_atencao_saude,
                id_plano,
                NULL AS idade_beneficiario,
                faixa_etaria,
                sexo,
                cd_municipio_beneficiario,
                porte,
                cd_modalidade,
                nm_modalidade,
                cd_municipio_prestador,
                uf_prestador,
                dt_realizacao,
                cd_carater_atendimento,
                tipo_atendimento,
                cd_motivo_saida,
                cbo,
                tipo_consulta,
                ind_acidente_doenca,
                lg_valor_preestabelecido,
                NULL AS lg_outlier,
                SUBSTRING(dt_realizacao, 1, 4) AS ano_realizacao
        '''
        s3 = f' FROM dtiss2.{uf_atual}_{anomes_atual}_amb_cons '
        executar_sql(s1 + s2 + s3)

        # Drop tabela antiga
        executar_sql(f'DROP TABLE dtiss2.{uf_atual}_{anomes_atual}_amb_cons')

        # Renomear nova tabela
        executar_sql(f'ALTER TABLE dtiss2.{uf_atual}_{anomes_atual}_amb_cons_1 RENAME TO {uf_atual}_{anomes_atual}_amb_cons')

        print(f'✔️ Tabela dtiss2.{uf_atual}_{anomes_atual} atualizada com sucesso.')


*2.4 MERGE BASE AMBULATORIAL - DET E AMB AND SELECIONANDO PROCEDIMENTOS*

In [7]:
# Parâmetros
n = anomes.shape[0]
m = uf.shape[0]

for i in range(0,n):      # anomes
    for j in range(0,1):  # uf
        uf_atual = uf.iloc[j, 0]
        anomes_atual = anomes.iloc[i, 0]
       ####----juntando as bases det com cons e selecionando os procedimentos por ano
        s1 = f'create table dtiss2.{uf_atual}_amb_{anomes_atual}  as '
        s2 = '''
        select
                   cons.id_evento_atencao_saude,cons.id_plano,cons.idade_beneficiario,cons.faixa_etaria,cons.sexo,cons.cd_municipio_beneficiario,cons.porte,cons.cd_modalidade,cons.nm_modalidade,cons.cd_municipio_prestador,cons.uf_prestador,cons.dt_realizacao,cons.cd_carater_atendimento,cons.tipo_atendimento,cons.cd_motivo_saida,cons.cbo,cons.tipo_consulta,cons.ind_acidente_doenca,cons.lg_valor_preestabelecido,cons.ano_realizacao,
    det.cd_procedimento,det.cd_tabela_referencia,det.qt_procedimento,det.vl_procedimento,det.vl_procedimento_fornecedor,det.ind_pacote,det.ind_tabela_propria,det.lg_outlier 
    ''' 
        s3 = f' from dtiss2.{uf_atual}_{anomes_atual}_amb_det det '
        s4 = f'inner join dtiss2.{uf_atual}_{anomes_atual}_amb_cons cons '
        s5 = '''
    on (det.id_evento_atencao_saude = cons.id_evento_atencao_saude)  
    and  (det.uf_prestador = cons.uf_prestador) 
    and  (det.dt_realizacao = cons.dt_realizacao) 
    and  (det.ano_realizacao = cons.ano_realizacao) 
    '''
        sintaxe = s1 + s2 + s3 + s4 + s5 
        sql = ( sintaxe )
        executar_sql (sql)
       ##################################----selecionando os procedimentos
        s1 = f'create table dtiss2.{uf_atual}_amb_1_{anomes_atual}  as '
        s2 = f' select *from dtiss2.{uf_atual}_amb_{anomes_atual} '       
        s3 = '''
      WHERE 
      cd_procedimento IN (
    '10101012',
    '10101039',
    '10106014',
    '10106030',
    '10106049',
    '10106090',
    '10106146',
    '20101023',
    '20101074',
    '20101082',
    '20101090',
    '20101104',
    '20101201',
    '20101236',
    '20102011',
    '20102020',
    '20102038',
    '20102070',
    '20103018',
    '20103026',
    '20103034',
    '20103042',
    '20103050',
    '20103069',
    '20103077',
    '20103093',
    '20103107',
    '20103131',
    '20103140',
    '20103182',
    '20103190',
    '20103204',
    '20103212',
    '20103220',
    '20103239',
    '20103247',
    '20103255',
    '20103263',
    '20103271',
    '20103280',
    '20103298',
    '20103301',
    '20103310',
    '20103328',
    '20103344',
    '20103360',
    '20103379',
    '20103387',
    '20103395',
    '20103409',
    '20103417',
    '20103425',
    '20103433',
    '20103441',
    '20103450',
    '20103468',
    '20103476',
    '20103484',
    '20103492',
    '20103506',
    '20103514',
    '20103522',
    '20103530',
    '20103565',
    '20103611',
    '20103620',
    '20103638',
    '20103646',
    '20103654',
    '20103662',
    '20103670',
    '20103689',
    '20103697',
    '20103700',
    '20103719',
    '20103727',
    '20103743',
    '20104014',
    '20104022',
    '20104049',
    '20104057',
    '20104065',
    '20104073',
    '20104081',
    '20104090',
    '20104103',
    '20104111',
    '20104120',
    '20104138',
    '20104146',
    '20104154',
    '20104200',
    '20104219',
    '20104227',
    '20104235',
    '20104243',
    '20104251',
    '20104278',
    '20104286',
    '20104294',
    '20104308',
    '20104316',
    '20104324',
    '20104332',
    '20104383',
    '20104391',
    '20104421',
    '20104430',
    '20105037',
    '20201052',
    '20201087',
    '20202016',
    '20202024',
    '20202032',
    '20203012',
    '20203020',
    '20203047',
    '20203063',
    '20203071',
    '20204027',
    '20204035',
    '20204043',
    '20204159',
    '20204167',
    '20204175',
    '30101018',
    '30101050',
    '30101077',
    '30101085',
    '30101093',
    '30101107',
    '30101123',
    '30101204',
    '30101212',
    '30101220',
    '30101247',
    '30101255',
    '30101263',
    '30101280',
    '30101298',
    '30101352',
    '30101387',
    '30101441',
    '30101450',
    '30101468',
    '30101476',
    '30101484',
    '30101492',
    '30101506',
    '30101590',
    '30101603',
    '30101611',
    '30101620',
    '30101638',
    '30101646',
    '30101662',
    '30101670',
    '30101735',
    '30101786',
    '30101794',
    '30101824',
    '30101840',
    '30101913',
    '30101930',
    '30101948',
    '30101956',
    '30201012',
    '30201055',
    '30201063',
    '30202027',
    '30202159',
    '30203015',
    '30203031',
    '30204011',
    '30204038',
    '30204097',
    '30204100',
    '30205018',
    '30205077',
    '30205093',
    '30205107',
    '30207215',
    '30207223',
    '30207231',
    '30208041',
    '30210119',
    '30210127',
    '30211018',
    '30212103',
    '30212120',
    '30213010',
    '30214017',
    '30301017',
    '30301025',
    '30301033',
    '30301041',
    '30301050',
    '30301068',
    '30301076',
    '30301084',
    '30301114',
    '30301122',
    '30301130',
    '30301149',
    '30301157',
    '30301190',
    '30301203',
    '30301238',
    '30301246',
    '30301254',
    '30301262',
    '30303028',
    '30303044',
    '30303052',
    '30303060',
    '30303087',
    '30304016',
    '30304032',
    '30304040',
    '30304059',
    '30304083',
    '30304091',
    '30304105',
    '30305012',
    '30306019',
    '30306027',
    '30306035',
    '30306043',
    '30306060',
    '30306078',
    '30307082',
    '30307139',
    '30307147',
    '30308011',
    '30310016',
    '30310024',
    '30310067',
    '30310075',
    '30310083',
    '30310105',
    '30310113',
    '30311012',
    '30311055',
    '30312043',
    '30312124',
    '30312132',
    '30312159',
    '30313040',
    '30313066',
    '30401011',
    '30401038',
    '30401046',
    '30401054',
    '30401100',
    '30402018',
    '30402026',
    '30402034',
    '30402042',
    '30402050',
    '30402077',
    '30402093',
    '30403014',
    '30403103',
    '30403162',
    '30501016',
    '30501024',
    '30501059',
    '30501067',
    '30501075',
    '30501083',
    '30501091',
    '30501113',
    '30501121',
    '30501130',
    '30501156',
    '30501164',
    '30501172',
    '30501199',
    '30501237',
    '30501288',
    '30501377',
    '30501440',
    '30501474',
    '30501482',
    '30501547',
    '30502160',
    '30502179',
    '30502195',
    '30601223',
    '30601231',
    '30601266',
    '30601304',
    '30601312',
    '30602017',
    '30602025',
    '30602050',
    '30602068',
    '30602181',
    '30602335',
    '30710014',
    '30710022',
    '30710030',
    '30710057',
    '30711010',
    '30711029',
    '30711037',
    '30712017',
    '30712025',
    '30712033',
    '30712041',
    '30712050',
    '30712068',
    '30712076',
    '30712084',
    '30712092',
    '30712106',
    '30712114',
    '30712122',
    '30712130',
    '30712149',
    '30713021',
    '30713030',
    '30713137',
    '30713145',
    '30714028',
    '30715040',
    '30715121',
    '30715130',
    '30715156',
    '30715237',
    '30715253',
    '30715407',
    '30717086',
    '30717094',
    '30717175',
    '30718040',
    '30718066',
    '30718104',
    '30719054',
    '30719070',
    '30719097',
    '30719119',
    '30720044',
    '30720087',
    '30720109',
    '30721091',
    '30721105',
    '30721130',
    '30721156',
    '30721172',
    '30721180',
    '30722012',
    '30722039',
    '30722160',
    '30722241',
    '30722292',
    '30722349',
    '30722357',
    '30722365',
    '30722373',
    '30722381',
    '30722403',
    '30722446',
    '30722497',
    '30722527',
    '30722535',
    '30722551',
    '30722586',
    '30722632',
    '30722640',
    '30722691',
    '30722713',
    '30722764',
    '30722900',
    '30722918',
    '30723035',
    '30723060',
    '30723094',
    '30723108',
    '30723116',
    '30723124',
    '30724163',
    '30725062',
    '30725100',
    '30725178',
    '30725186',
    '30725194',
    '30725208',
    '30726050',
    '30726085',
    '30726115',
    '30726174',
    '30727057',
    '30727103',
    '30727120',
    '30727146',
    '30727197',
    '30728088',
    '30728096',
    '30728118',
    '30728134',
    '30729068',
    '30729114',
    '30729130',
    '30729149',
    '30729165',
    '30729262',
    '30729343',
    '30730023',
    '30730112',
    '30730163',
    '30730171',
    '30731011',
    '30731020',
    '30731046',
    '30731089',
    '30731186',
    '30731194',
    '30731259',
    '30801044',
    '30801184',
    '30803241',
    '30804019',
    '30804086',
    '30804116',
    '30805023',
    '30904013',
    '30907071',
    '30908027',
    '30908078',
    '30909023',
    '30909031',
    '30909139',
    '30909147',
    '30911176',
    '30912156',
    '30913012',
    '30913071',
    '30913080',
    '30913098',
    '30913128',
    '30913144',
    '30913152',
    '30914124',
    '30918014',
    '30918022',
    '30918030',
    '31003362',
    '31004016',
    '31004024',
    '31004032',
    '31004040',
    '31004067',
    '31004180',
    '31004199',
    '31004229',
    '31004253',
    '31004318',
    '31005071',
    '31005080',
    '31005675',
    '31005691',
    '31007015',
    '31008011',
    '31008020',
    '31008038',
    '31009018',
    '31009026',
    '31009247',
    '31009352',
    '31101089',
    '31101313',
    '31101402',
    '31102026',
    '31103030',
    '31103057',
    '31103189',
    '31103197',
    '31103235',
    '31103391',
    '31103405',
    '31103430',
    '31103596',
    '31104010',
    '31104029',
    '31104045',
    '31104061',
    '31104142',
    '31104169',
    '31202012',
    '31202020',
    '31202047',
    '31203027',
    '31203086',
    '31204015',
    '31204023',
    '31204066',
    '31205011',
    '31206034',
    '31206174',
    '31206204',
    '31206212',
    '31206220',
    '31301010',
    '31301029',
    '31301045',
    '31301070',
    '31301088',
    '31301096',
    '31301100',
    '31301118',
    '31302017',
    '31302076',
    '31302084',
    '31302114',
    '31303021',
    '31303030',
    '31303064',
    '31303072',
    '31303170',
    '31303196',
    '31303269',
    '31303293',
    '31303307',
    '31306047',
    '31401350',
    '31402038',
    '31403018',
    '31403026',
    '31403301',
    '31403328',
    '31405010',
    '31601014',
    '31602029',
    '31602070',
    '31602118',
    '31602126',
    '31602185',
    '31602207',
    '31602223',
    '31602231',
    '31602240',
    '31602258',
    '31602266',
    '31602274',
    '31602282',
    '31602290',
    '31602304',
    '31602312',
    '31602320',
    '31602339',
    '40101010',
    '40101029',
    '40101037',
    '40101045',
    '40101053',
    '40101061',
    '40102025',
    '40102033',
    '40102041',
    '40102050',
    '40102068',
    '40102076',
    '40102084',
    '40102092',
    '40102106',
    '40102122',
    '40102130',
    '40103013',
    '40103064',
    '40103072',
    '40103080',
    '40103099',
    '40103102',
    '40103110',
    '40103137',
    '40103161',
    '40103170',
    '40103196',
    '40103234',
    '40103242',
    '40103250',
    '40103269',
    '40103285',
    '40103307',
    '40103315',
    '40103323',
    '40103331',
    '40103340',
    '40103358',
    '40103366',
    '40103374',
    '40103382',
    '40103390',
    '40103404',
    '40103412',
    '40103439',
    '40103447',
    '40103455',
    '40103463',
    '40103498',
    '40103501',
    '40103510',
    '40103560',
    '40103579',
    '40103609',
    '40103617',
    '40103625',
    '40103633',
    '40103641',
    '40103650',
    '40103668',
    '40103714',
    '40103722',
    '40103730',
    '40103749',
    '40103757',
    '40103803',
    '40103820',
    '40103862',
    '40103889',
    '40103897',
    '40104010',
    '40104028',
    '40104036',
    '40104125',
    '40105016',
    '40105024',
    '40105032',
    '40105040',
    '40105059',
    '40105067',
    '40105075',
    '40105083',
    '40105091',
    '40105148',
    '40201015',
    '40201023',
    '40201058',
    '40201066',
    '40201074',
    '40201082',
    '40201090',
    '40201104',
    '40201112',
    '40201120',
    '40201139',
    '40201147',
    '40201155',
    '40201171',
    '40201180',
    '40201198',
    '40201201',
    '40201210',
    '40201228',
    '40201236',
    '40201244',
    '40201252',
    '40201260',
    '40201309',
    '40201333',
    '40201350',
    '40202038',
    '40202089',
    '40202097',
    '40202135',
    '40202143',
    '40202178',
    '40202186',
    '40202194',
    '40202240',
    '40202259',
    '40202267',
    '40202330',
    '40202348',
    '40202410',
    '40202429',
    '40202437',
    '40202445',
    '40202453',
    '40202488',
    '40202496',
    '40202518',
    '40202534',
    '40202542',
    '40202550',
    '40202569',
    '40202577',
    '40202585',
    '40202593',
    '40202607',
    '40202615',
    '40202623',
    '40202666',
    '40202674',
    '40202682',
    '40202690',
    '40202704',
    '40202712',
    '40202720',
    '40202739',
    '40202747',
    '40202763',
    '40301010',
    '40301028',
    '40301036',
    '40301044',
    '40301060',
    '40301087',
    '40301109',
    '40301117',
    '40301125',
    '40301133',
    '40301141',
    '40301150',
    '40301168',
    '40301184',
    '40301192',
    '40301206',
    '40301214',
    '40301222',
    '40301230',
    '40301249',
    '40301257',
    '40301265',
    '40301273',
    '40301281',
    '40301290',
    '40301303',
    '40301311',
    '40301320',
    '40301346',
    '40301354',
    '40301362',
    '40301370',
    '40301389',
    '40301397',
    '40301400',
    '40301419',
    '40301427',
    '40301435',
    '40301443',
    '40301451',
    '40301460',
    '40301478',
    '40301486',
    '40301494',
    '40301508',
    '40301516',
    '40301524',
    '40301540',
    '40301559',
    '40301567',
    '40301583',
    '40301591',
    '40301605',
    '40301621',
    '40301630',
    '40301648',
    '40301656',
    '40301664',
    '40301672',
    '40301680',
    '40301699',
    '40301702',
    '40301729',
    '40301737',
    '40301745',
    '40301753',
    '40301761',
    '40301770',
    '40301788',
    '40301796',
    '40301800',
    '40301818',
    '40301826',
    '40301834',
    '40301842',
    '40301850',
    '40301869',
    '40301877',
    '40301885',
    '40301893',
    '40301907',
    '40301915',
    '40301923',
    '40301931',
    '40301940',
    '40301958',
    '40301966',
    '40301974',
    '40301982',
    '40301990',
    '40302016',
    '40302024',
    '40302032',
    '40302040',
    '40302059',
    '40302067',
    '40302075',
    '40302083',
    '40302091',
    '40302105',
    '40302113',
    '40302121',
    '40302130',
    '40302156',
    '40302164',
    '40302180',
    '40302199',
    '40302229',
    '40302237',
    '40302245',
    '40302270',
    '40302288',
    '40302296',
    '40302300',
    '40302318',
    '40302326',
    '40302334',
    '40302342',
    '40302350',
    '40302377',
    '40302385',
    '40302393',
    '40302407',
    '40302415',
    '40302423',
    '40302431',
    '40302458',
    '40302474',
    '40302482',
    '40302490',
    '40302504',
    '40302512',
    '40302520',
    '40302539',
    '40302547',
    '40302555',
    '40302563',
    '40302571',
    '40302580',
    '40302598',
    '40302601',
    '40302610',
    '40302628',
    '40302636',
    '40302644',
    '40302652',
    '40302679',
    '40302695',
    '40302709',
    '40302717',
    '40302725',
    '40302733',
    '40302741',
    '40302750',
    '40302768',
    '40302776',
    '40302830',
    '40302881',
    '40302890',
    '40302903',
    '40302946',
    '40303012',
    '40303020',
    '40303039',
    '40303055',
    '40303063',
    '40303071',
    '40303080',
    '40303098',
    '40303101',
    '40303110',
    '40303128',
    '40303136',
    '40303144',
    '40303152',
    '40303160',
    '40303179',
    '40303187',
    '40303250',
    '40303268',
    '40304019',
    '40304027',
    '40304035',
    '40304043',
    '40304051',
    '40304060',
    '40304078',
    '40304086',
    '40304094',
    '40304108',
    '40304116',
    '40304132',
    '40304140',
    '40304159',
    '40304167',
    '40304175',
    '40304183',
    '40304191',
    '40304205',
    '40304213',
    '40304221',
    '40304230',
    '40304248',
    '40304256',
    '40304264',
    '40304272',
    '40304280',
    '40304299',
    '40304302',
    '40304310',
    '40304337',
    '40304345',
    '40304353',
    '40304361',
    '40304370',
    '40304388',
    '40304418',
    '40304434',
    '40304450',
    '40304469',
    '40304477',
    '40304485',
    '40304493',
    '40304507',
    '40304515',
    '40304523',
    '40304531',
    '40304540',
    '40304558',
    '40304566',
    '40304574',
    '40304582',
    '40304590',
    '40304612',
    '40304620',
    '40304639',
    '40304647',
    '40304655',
    '40304671',
    '40304680',
    '40304698',
    '40304701',
    '40304710',
    '40304728',
    '40304736',
    '40304752',
    '40304760',
    '40304787',
    '40304809',
    '40304817',
    '40304825',
    '40304850',
    '40304876',
    '40304884',
    '40304892',
    '40304906',
    '40304914',
    '40304922',
    '40304930',
    '40304949',
    '40304973',
    '40305015',
    '40305066',
    '40305074',
    '40305082',
    '40305090',
    '40305112',
    '40305120',
    '40305163',
    '40305210',
    '40305228',
    '40305236',
    '40305279',
    '40305287',
    '40305295',
    '40305341',
    '40305368',
    '40305384',
    '40305406',
    '40305449',
    '40305465',
    '40305503',
    '40305546',
    '40305554',
    '40305562',
    '40305570',
    '40305589',
    '40305597',
    '40305627',
    '40305740',
    '40305767',
    '40305775',
    '40305783',
    '40306011',
    '40306020',
    '40306046',
    '40306054',
    '40306062',
    '40306070',
    '40306089',
    '40306097',
    '40306100',
    '40306119',
    '40306127',
    '40306135',
    '40306143',
    '40306151',
    '40306160',
    '40306194',
    '40306208',
    '40306216',
    '40306224',
    '40306232',
    '40306240',
    '40306259',
    '40306267',
    '40306275',
    '40306283',
    '40306291',
    '40306305',
    '40306313',
    '40306330',
    '40306348',
    '40306356',
    '40306364',
    '40306372',
    '40306380',
    '40306399',
    '40306402',
    '40306410',
    '40306429',
    '40306437',
    '40306445',
    '40306453',
    '40306461',
    '40306470',
    '40306488',
    '40306496',
    '40306500',
    '40306518',
    '40306534',
    '40306542',
    '40306550',
    '40306593',
    '40306607',
    '40306615',
    '40306623',
    '40306631',
    '40306640',
    '40306658',
    '40306666',
    '40306674',
    '40306682',
    '40306690',
    '40306704',
    '40306712',
    '40306739',
    '40306747',
    '40306755',
    '40306763',
    '40306771',
    '40306780',
    '40306798',
    '40306801',
    '40306810',
    '40306852',
    '40306860',
    '40306879',
    '40306887',
    '40306895',
    '40306909',
    '40306917',
    '40306925',
    '40306933',
    '40306941',
    '40306950',
    '40306968',
    '40306976',
    '40306984',
    '40306992',
    '40307018',
    '40307026',
    '40307034',
    '40307042',
    '40307050',
    '40307069',
    '40307077',
    '40307085',
    '40307093',
    '40307107',
    '40307115',
    '40307123',
    '40307140',
    '40307158',
    '40307166',
    '40307174',
    '40307182',
    '40307190',
    '40307204',
    '40307212',
    '40307220',
    '40307247',
    '40307255',
    '40307263',
    '40307271',
    '40307280',
    '40307298',
    '40307301',
    '40307336',
    '40307344',
    '40307352',
    '40307387',
    '40307395',
    '40307409',
    '40307417',
    '40307425',
    '40307433',
    '40307441',
    '40307450',
    '40307468',
    '40307476',
    '40307484',
    '40307492',
    '40307522',
    '40307530',
    '40307565',
    '40307573',
    '40307581',
    '40307603',
    '40307611',
    '40307620',
    '40307638',
    '40307689',
    '40307697',
    '40307700',
    '40307719',
    '40307727',
    '40307735',
    '40307743',
    '40307751',
    '40307760',
    '40307794',
    '40307808',
    '40307824',
    '40307832',
    '40307840',
    '40307859',
    '40307867',
    '40307875',
    '40307883',
    '40307905',
    '40307948',
    '40307964',
    '40307972',
    '40307999',
    '40308014',
    '40308022',
    '40308030',
    '40308090',
    '40308120',
    '40308138',
    '40308154',
    '40308162',
    '40308170',
    '40308235',
    '40308286',
    '40308294',
    '40308308',
    '40308316',
    '40308340',
    '40308359',
    '40308367',
    '40308383',
    '40308391',
    '40308405',
    '40308413',
    '40308421',
    '40308529',
    '40308553',
    '40308804',
    '40308901',
    '40309010',
    '40309029',
    '40309037',
    '40309045',
    '40309053',
    '40309061',
    '40309070',
    '40309088',
    '40309096',
    '40309100',
    '40309118',
    '40309126',
    '40309134',
    '40309142',
    '40309150',
    '40309169',
    '40309266',
    '40309304',
    '40309312',
    '40309320',
    '40309401',
    '40309410',
    '40309428',
    '40309436',
    '40309444',
    '40309509',
    '40309517',
    '40309525',
    '40310019',
    '40310035',
    '40310043',
    '40310051',
    '40310060',
    '40310078',
    '40310086',
    '40310094',
    '40310108',
    '40310116',
    '40310124',
    '40310132',
    '40310140',
    '40310159',
    '40310167',
    '40310175',
    '40310183',
    '40310191',
    '40310205',
    '40310213',
    '40310221',
    '40310230',
    '40310248',
    '40310256',
    '40310264',
    '40310272',
    '40310280',
    '40310299',
    '40310302',
    '40310310',
    '40310329',
    '40310337',
    '40310345',
    '40310361',
    '40310370',
    '40310388',
    '40310400',
    '40310418',
    '40310426',
    '40310434',
    '40310515',
    '40310540',
    '40310558',
    '40310566',
    '40310590',
    '40310604',
    '40310612',
    '40310620',
    '40310647',
    '40310671',
    '40310728',
    '40310736',
    '40311015',
    '40311023',
    '40311031',
    '40311040',
    '40311058',
    '40311066',
    '40311074',
    '40311082',
    '40311090',
    '40311104',
    '40311112',
    '40311120',
    '40311139',
    '40311147',
    '40311155',
    '40311163',
    '40311171',
    '40311180',
    '40311198',
    '40311201',
    '40311210',
    '40311228',
    '40311236',
    '40311244',
    '40311252',
    '40311279',
    '40311295',
    '40311309',
    '40311317',
    '40311325',
    '40311341',
    '40311350',
    '40311368',
    '40311392',
    '40311430',
    '40311465',
    '40311473',
    '40311503',
    '40312020',
    '40312046',
    '40312054',
    '40312062',
    '40312070',
    '40312097',
    '40312100',
    '40312127',
    '40312143',
    '40312151',
    '40312160',
    '40312178',
    '40312194',
    '40312224',
    '40312232',
    '40312240',
    '40312259',
    '40312267',
    '40313018',
    '40313026',
    '40313034',
    '40313042',
    '40313050',
    '40313069',
    '40313077',
    '40313093',
    '40313107',
    '40313115',
    '40313123',
    '40313140',
    '40313158',
    '40313166',
    '40313174',
    '40313182',
    '40313190',
    '40313204',
    '40313212',
    '40313247',
    '40313263',
    '40313280',
    '40313301',
    '40313310',
    '40313328',
    '40313336',
    '40313344',
    '40314022',
    '40314030',
    '40314049',
    '40314057',
    '40314065',
    '40314081',
    '40314090',
    '40314103',
    '40314111',
    '40314120',
    '40314138',
    '40314146',
    '40314154',
    '40314162',
    '40314170',
    '40314197',
    '40314227',
    '40314235',
    '40314243',
    '40314251',
    '40314260',
    '40314278',
    '40314286',
    '40314294',
    '40314308',
    '40314359',
    '40314413',
    '40314430',
    '40314448',
    '40314502',
    '40314537',
    '40314545',
    '40314561',
    '40316017',
    '40316025',
    '40316033',
    '40316041',
    '40316050',
    '40316068',
    '40316076',
    '40316084',
    '40316092',
    '40316106',
    '40316114',
    '40316122',
    '40316130',
    '40316149',
    '40316157',
    '40316165',
    '40316173',
    '40316181',
    '40316190',
    '40316203',
    '40316211',
    '40316220',
    '40316238',
    '40316246',
    '40316254',
    '40316262',
    '40316270',
    '40316289',
    '40316297',
    '40316300',
    '40316319',
    '40316327',
    '40316335',
    '40316343',
    '40316351',
    '40316360',
    '40316378',
    '40316386',
    '40316394',
    '40316408',
    '40316416',
    '40316424',
    '40316432',
    '40316440',
    '40316459',
    '40316467',
    '40316475',
    '40316483',
    '40316491',
    '40316505',
    '40316513',
    '40316521',
    '40316530',
    '40316548',
    '40316556',
    '40316564',
    '40316572',
    '40316599',
    '40316602',
    '40316661',
    '40316769',
    '40316785',
    '40316831',
    '40316866',
    '40316874',
    '40316955',
    '40316963',
    '40317080',
    '40317129',
    '40317137',
    '40317145',
    '40317153',
    '40317161',
    '40317170',
    '40317188',
    '40317196',
    '40317200',
    '40317226',
    '40317250',
    '40317269',
    '40317277',
    '40317285',
    '40317293',
    '40317374',
    '40317390',
    '40317404',
    '40317412',
    '40317420',
    '40317439',
    '40317471',
    '40319032',
    '40319040',
    '40319091',
    '40319113',
    '40319121',
    '40319130',
    '40319148',
    '40319172',
    '40319199',
    '40319229',
    '40319253',
    '40319261',
    '40319270',
    '40319288',
    '40319296',
    '40319318',
    '40319326',
    '40319334',
    '40319369',
    '40319377',
    '40319385',
    '40319393',
    '40319407',
    '40319415',
    '40319431',
    '40319440',
    '40319458',
    '40319466',
    '40319474',
    '40321029',
    '40321096',
    '40321142',
    '40321207',
    '40321223',
    '40321231',
    '40321312',
    '40321347',
    '40321380',
    '40321410',
    '40321460',
    '40321479',
    '40321509',
    '40321517',
    '40321568',
    '40321614',
    '40321681',
    '40321690',
    '40321703',
    '40321711',
    '40321720',
    '40321738',
    '40321754',
    '40321762',
    '40321770',
    '40321789',
    '40321797',
    '40321800',
    '40321819',
    '40321827',
    '40321916',
    '40321967',
    '40321975',
    '40321983',
    '40322025',
    '40322041',
    '40322050',
    '40322068',
    '40322076',
    '40322084',
    '40322114',
    '40322157',
    '40322165',
    '40322173',
    '40322181',
    '40322190',
    '40322220',
    '40322246',
    '40322270',
    '40322289',
    '40322300',
    '40322319',
    '40322351',
    '40322360',
    '40322378',
    '40322386',
    '40322394',
    '40322408',
    '40322416',
    '40322432',
    '40322467',
    '40322475',
    '40322483',
    '40322491',
    '40322505',
    '40322564',
    '40323030',
    '40323048',
    '40323110',
    '40323153',
    '40323404',
    '40323471',
    '40323480',
    '40323510',
    '40323552',
    '40323595',
    '40323889',
    '40323897',
    '40323900',
    '40323919',
    '40323978',
    '40324052',
    '40324060',
    '40324079',
    '40324125',
    '40324176',
    '40324192',
    '40324265',
    '40324362',
    '40324370',
    '40324389',
    '40324559',
    '40324567',
    '40324591',
    '40324605',
    '40401014',
    '40401022',
    '40402010',
    '40402029',
    '40402037',
    '40402045',
    '40402053',
    '40402061',
    '40402070',
    '40402088',
    '40402096',
    '40402100',
    '40402118',
    '40402126',
    '40402134',
    '40402142',
    '40402150',
    '40402169',
    '40403025',
    '40403068',
    '40403092',
    '40403106',
    '40403122',
    '40403130',
    '40403149',
    '40403157',
    '40403165',
    '40403173',
    '40403181',
    '40403190',
    '40403203',
    '40403211',
    '40403220',
    '40403238',
    '40403246',
    '40403254',
    '40403262',
    '40403289',
    '40403300',
    '40403319',
    '40403327',
    '40403335',
    '40403343',
    '40403351',
    '40403360',
    '40403378',
    '40403386',
    '40403408',
    '40403416',
    '40403424',
    '40403440',
    '40403467',
    '40403483',
    '40403505',
    '40403521',
    '40403548',
    '40403564',
    '40403580',
    '40403602',
    '40403629',
    '40403645',
    '40403661',
    '40403688',
    '40403696',
    '40403700',
    '40403718',
    '40403750',
    '40403769',
    '40403777',
    '40403840',
    '40403890',
    '40403920',
    '40403939',
    '40403947',
    '40403955',
    '40403963',
    '40403971',
    '40403980',
    '40403998',
    '40404072',
    '40404129',
    '40404404',
    '40404536',
    '40501019',
    '40501027',
    '40501035',
    '40501043',
    '40501051',
    '40501060',
    '40501078',
    '40501086',
    '40501094',
    '40501108',
    '40501116',
    '40501124',
    '40501132',
    '40501159',
    '40501167',
    '40501175',
    '40501183',
    '40501191',
    '40501205',
    '40501213',
    '40501221',
    '40501230',
    '40501248',
    '40501256',
    '40501264',
    '40501272',
    '40501280',
    '40502015',
    '40502040',
    '40502058',
    '40502066',
    '40502074',
    '40502082',
    '40502090',
    '40502104',
    '40502112',
    '40502120',
    '40502139',
    '40502147',
    '40502155',
    '40502163',
    '40502171',
    '40502180',
    '40502198',
    '40502201',
    '40502228',
    '40502236',
    '40502244',
    '40503011',
    '40503020',
    '40503038',
    '40503046',
    '40503054',
    '40503062',
    '40503089',
    '40503097',
    '40503100',
    '40503119',
    '40503127',
    '40503143',
    '40503151',
    '40503160',
    '40503178',
    '40503186',
    '40503194',
    '40503208',
    '40503216',
    '40503232',
    '40503240',
    '40503259',
    '40503267',
    '40503275',
    '40503283',
    '40503313',
    '40503348',
    '40503356',
    '40503364',
    '40503372',
    '40503380',
    '40503399',
    '40503402',
    '40503410',
    '40503429',
    '40503437',
    '40503445',
    '40503453',
    '40503461',
    '40503470',
    '40503488',
    '40503496',
    '40503500',
    '40503518',
    '40503526',
    '40503534',
    '40503542',
    '40503577',
    '40503585',
    '40503593',
    '40503607',
    '40503615',
    '40503623',
    '40503631',
    '40503640',
    '40503658',
    '40503674',
    '40503682',
    '40503690',
    '40503712',
    '40503739',
    '40503747',
    '40503755',
    '40503763',
    '40503771',
    '40503780',
    '40503798',
    '40503801',
    '40503828',
    '40503836',
    '40503844',
    '40601013',
    '40601021',
    '40601030',
    '40601072',
    '40601080',
    '40601099',
    '40601102',
    '40601110',
    '40601129',
    '40601137',
    '40601145',
    '40601153',
    '40601161',
    '40601170',
    '40601188',
    '40601196',
    '40601200',
    '40601218',
    '40601226',
    '40601234',
    '40601242',
    '40601250',
    '40601269',
    '40601277',
    '40601285',
    '40601293',
    '40601307',
    '40601323',
    '40601331',
    '40601340',
    '40601358',
    '40601366',
    '40601374',
    '40601382',
    '40601390',
    '40601404',
    '40601412',
    '40601420',
    '40601439',
    '40701018',
    '40701034',
    '40701042',
    '40701050',
    '40701069',
    '40701077',
    '40701085',
    '40701093',
    '40701107',
    '40701115',
    '40701123',
    '40701131',
    '40701140',
    '40701158',
    '40702014',
    '40702022',
    '40702030',
    '40702049',
    '40702057',
    '40702065',
    '40702073',
    '40702081',
    '40702090',
    '40702103',
    '40702111',
    '40702146',
    '40703010',
    '40703029',
    '40703037',
    '40703045',
    '40703053',
    '40703061',
    '40703070',
    '40703088',
    '40703096',
    '40703100',
    '40704017',
    '40704025',
    '40704033',
    '40704041',
    '40704050',
    '40704068',
    '40704076',
    '40704084',
    '40704092',
    '40705013',
    '40705021',
    '40705030',
    '40705048',
    '40705056',
    '40705064',
    '40706010',
    '40706028',
    '40707016',
    '40707032',
    '40707040',
    '40707059',
    '40707067',
    '40707075',
    '40707083',
    '40708012',
    '40708020',
    '40708039',
    '40708047',
    '40708063',
    '40708071',
    '40708080',
    '40708098',
    '40708101',
    '40708110',
    '40708128',
    '40709019',
    '40709027',
    '40709035',
    '40710017',
    '40710025',
    '40710033',
    '40710041',
    '40710050',
    '40710068',
    '40710076',
    '40710084',
    '40710092',
    '40710106',
    '40711021',
    '40801012',
    '40801020',
    '40801039',
    '40801047',
    '40801055',
    '40801063',
    '40801071',
    '40801080',
    '40801098',
    '40801101',
    '40801110',
    '40801128',
    '40801136',
    '40801160',
    '40801195',
    '40801209',
    '40802019',
    '40802027',
    '40802035',
    '40802043',
    '40802051',
    '40802060',
    '40802078',
    '40802086',
    '40802094',
    '40802108',
    '40802116',
    '40803015',
    '40803023',
    '40803031',
    '40803040',
    '40803058',
    '40803066',
    '40803074',
    '40803082',
    '40803090',
    '40803104',
    '40803112',
    '40803120',
    '40803139',
    '40803147',
    '40803155',
    '40804011',
    '40804020',
    '40804038',
    '40804046',
    '40804054',
    '40804062',
    '40804070',
    '40804089',
    '40804097',
    '40804100',
    '40804119',
    '40804127',
    '40804135',
    '40805018',
    '40805026',
    '40805034',
    '40805042',
    '40805050',
    '40805069',
    '40805077',
    '40806014',
    '40806022',
    '40806030',
    '40806049',
    '40806057',
    '40806065',
    '40806073',
    '40806081',
    '40806090',
    '40806103',
    '40806111',
    '40806120',
    '40806189',
    '40806200',
    '40807010',
    '40807029',
    '40807037',
    '40807045',
    '40807053',
    '40807061',
    '40807070',
    '40807088',
    '40807096',
    '40808017',
    '40808025',
    '40808033',
    '40808041',
    '40808050',
    '40808114',
    '40808122',
    '40808130',
    '40808149',
    '40808157',
    '40808165',
    '40808190',
    '40808203',
    '40808211',
    '40808220',
    '40808238',
    '40808246',
    '40808254',
    '40808262',
    '40808270',
    '40808289',
    '40808297',
    '40809021',
    '40809030',
    '40809048',
    '40809056',
    '40809064',
    '40809072',
    '40809080',
    '40809102',
    '40809153',
    '40809161',
    '40809170',
    '40809188',
    '40810011',
    '40810020',
    '40810046',
    '40811018',
    '40811026',
    '40812022',
    '40812030',
    '40812049',
    '40812057',
    '40812073',
    '40812081',
    '40812090',
    '40812103',
    '40812111',
    '40812120',
    '40812138',
    '40812146',
    '40813053',
    '40813363',
    '40813410',
    '40813428',
    '40813436',
    '40813444',
    '40813452',
    '40813460',
    '40813479',
    '40813487',
    '40813495',
    '40813509',
    '40813517',
    '40813525',
    '40813533',
    '40813860',
    '40814092',
    '40814114',
    '40814130',
    '40814149',
    '40901017',
    '40901025',
    '40901033',
    '40901041',
    '40901050',
    '40901068',
    '40901076',
    '40901084',
    '40901092',
    '40901106',
    '40901114',
    '40901122',
    '40901130',
    '40901149',
    '40901173',
    '40901181',
    '40901190',
    '40901203',
    '40901211',
    '40901220',
    '40901238',
    '40901246',
    '40901254',
    '40901262',
    '40901270',
    '40901289',
    '40901297',
    '40901300',
    '40901319',
    '40901335',
    '40901351',
    '40901360',
    '40901378',
    '40901386',
    '40901394',
    '40901408',
    '40901416',
    '40901424',
    '40901432',
    '40901440',
    '40901459',
    '40901467',
    '40901475',
    '40901483',
    '40901505',
    '40901513',
    '40901521',
    '40901530',
    '40901548',
    '40901556',
    '40901572',
    '40901602',
    '40901610',
    '40901629',
    '40901637',
    '40901645',
    '40901653',
    '40901661',
    '40901696',
    '40901700',
    '40901718',
    '40901734',
    '40901742',
    '40901750',
    '40901769',
    '40902013',
    '40902021',
    '40902072',
    '40902080',
    '40902110',
    '40902129',
    '40902137',
    '41001010',
    '41001028',
    '41001036',
    '41001044',
    '41001052',
    '41001060',
    '41001079',
    '41001095',
    '41001109',
    '41001117',
    '41001125',
    '41001133',
    '41001141',
    '41001150',
    '41001176',
    '41001184',
    '41001222',
    '41001230',
    '41001273',
    '41001281',
    '41001338',
    '41001370',
    '41001389',
    '41001397',
    '41001400',
    '41001419',
    '41001427',
    '41001435',
    '41001443',
    '41001451',
    '41001460',
    '41001478',
    '41001516',
    '41001524',
    '41001532',
    '41002016',
    '41002032',
    '41002040',
    '41002059',
    '41101014',
    '41101022',
    '41101030',
    '41101057',
    '41101065',
    '41101073',
    '41101081',
    '41101090',
    '41101103',
    '41101111',
    '41101120',
    '41101138',
    '41101146',
    '41101154',
    '41101170',
    '41101189',
    '41101197',
    '41101200',
    '41101219',
    '41101227',
    '41101235',
    '41101243',
    '41101251',
    '41101260',
    '41101278',
    '41101286',
    '41101294',
    '41101308',
    '41101316',
    '41101332',
    '41101340',
    '41101359',
    '41101472',
    '41101480',
    '41101499',
    '41101502',
    '41101510',
    '41101529',
    '41101537',
    '41101545',
    '41101553',
    '41101596',
    '41101600',
    '41101618',
    '41101626',
    '41101669',
    '41102010',
    '41203011',
    '41203020',
    '41203038',
    '41203046',
    '41203054',
    '41203062',
    '41203070',
    '41203089',
    '41203097',
    '41203100',
    '41203119',
    '41203127',
    '41203135',
    '41203143',
    '41203151',
    '41203208',
    '41204018',
    '41204026',
    '41204034',
    '41204042',
    '41204050',
    '41204069',
    '41204077',
    '41204085',
    '41204093',
    '41204107',
    '41205014',
    '41205022',
    '41205030',
    '41205073',
    '41205120',
    '41206010',
    '41206037',
    '41206045',
    '41206053',
    '41206061',
    '41206070',
    '41301013',
    '41301021',
    '41301030',
    '41301048',
    '41301056',
    '41301064',
    '41301072',
    '41301080',
    '41301099',
    '41301102',
    '41301110',
    '41301129',
    '41301137',
    '41301145',
    '41301153',
    '41301161',
    '41301170',
    '41301188',
    '41301200',
    '41301218',
    '41301226',
    '41301234',
    '41301242',
    '41301250',
    '41301269',
    '41301277',
    '41301285',
    '41301307',
    '41301315',
    '41301323',
    '41301331',
    '41301340',
    '41301358',
    '41301366',
    '41301374',
    '41301382',
    '41301390',
    '41301420',
    '41301439',
    '41301447',
    '41301463',
    '41301471',
    '41301498',
    '41301536',
    '41301544',
    '41301552',
    '41401018',
    '41401026',
    '41401042',
    '41401069',
    '41401077',
    '41401085',
    '41401107',
    '41401131',
    '41401166',
    '41401174',
    '41401182',
    '41401190',
    '41401204',
    '41401212',
    '41401220',
    '41401239',
    '41401247',
    '41401255',
    '41401263',
    '41401271',
    '41401298',
    '41401301',
    '41401360',
    '41401379',
    '41401387',
    '41401395',
    '41401409',
    '41401425',
    '41401433',
    '41401441',
    '41401450',
    '41401468',
    '41401476',
    '41401484',
    '41401492',
    '41401514',
    '41401522',
    '41401530',
    '41401557',
    '41401581',
    '41401646',
    '41401751',
    '41401760',
    '41401778',
    '41401786',
    '41401794',
    '41401808',
    '41501012',
    '41501020',
    '41501047',
    '41501063',
    '41501071',
    '41501080',
    '41501098',
    '41501101',
    '41501128',
    '41501144',
    '41501195',
    '41501209',
    '41501225',
    '41501233',
    '41501268',
    '50000012',
    '50000055',
    '50000071',
    '50000080',
    '50000101',
    '50000136',
    '50000144',
    '50000160',
    '50000195',
    '50000209',
    '50000217',
    '50000233',
    '50000349',
    '50000365',
    '50000381',
    '50000390',
    '50000403',
    '50000411',
    '50000420',
    '50000454',
    '50000462',
    '50000470',
    '50000527',
    '50000560',
    '50000586',
    '50000608',
    '50000616',
    '50000632',
    '50000659',
    '50000675',
    '50000691',
    '50000713',
    '50000721',
    '50000730',
    '50000748',
    '50000756',
    '50000764',
    '50000772',
    '50000780',
    '50000799',
    '50000802',
    '50000810',
    '50000829',
    '50000900',
    '50000926',
    '50000934',
    '50001000',
    '50001019',
    '50001043',
    '50001051',
    '50001060',
    '50001078',
    '81000111',
    '81000138',
    '81000154',
    '81000170',
    '81000324',
    '81000340',
    '81000367',
    '81000405',
    '81000430',
    '81000472',
    '81000561',
    '81000570',
    '82000239',
    '82000247',
    '82000255',
    '82000263',
    '82000271',
    '82000280',
    '82000298',
    '82000301',
    '82000743',
    '82000778',
    '82000786',
    '82000794',
    '82000808',
    '82000883',
    '82000891',
    '82000905',
    '82000913',
    '82001022',
    '82001030',
    '82001120',
    '82001197',
    '82001219',
    '82001235',
    '82001367',
    '82001375',
    '82001430',
    '82001499',
    '82001510',
    '82001529',
    '82001545',
    '82001553',
    '82001588',
    '82001596',
    '82001618',
    '82001634',
    '82001642',
    '82001758',
    '20104464',
    '20204183',
    '30304156',
    '40503852',
    '40901793',
    '40707091',
    '001',
    '002',
    '003',
    '004',
    '005',
    '006',
    '007',
    '008',
    '009',
    '010',
    '011',
    '012',
    '013',
    '014',
    '015',
    '016',
    '017',
    '018',
    '019',
    '020',
    '021',
    '022',
    '023',
    '024',
    '025',
    '026',
    '027',
    '028',
    '029',
    '030',
    '031'
    );    
        '''
    sintaxe = s1 + s2 + s3 
    sql = ( sintaxe )
    executar_sql(sql)
    print(f'✔️ Tabela dtiss2.{uf_atual}_{anomes_atual} feito merge e procedimentos separados com sucesso.')

✔️ Tabela dtiss2.RO_202301 feito merge e procedimentos separados com sucesso.
✔️ Tabela dtiss2.RO_202302 feito merge e procedimentos separados com sucesso.
✔️ Tabela dtiss2.RO_202303 feito merge e procedimentos separados com sucesso.
✔️ Tabela dtiss2.RO_202304 feito merge e procedimentos separados com sucesso.
✔️ Tabela dtiss2.RO_202305 feito merge e procedimentos separados com sucesso.
✔️ Tabela dtiss2.RO_202306 feito merge e procedimentos separados com sucesso.
✔️ Tabela dtiss2.RO_202307 feito merge e procedimentos separados com sucesso.
✔️ Tabela dtiss2.RO_202308 feito merge e procedimentos separados com sucesso.
✔️ Tabela dtiss2.RO_202309 feito merge e procedimentos separados com sucesso.
✔️ Tabela dtiss2.RO_202310 feito merge e procedimentos separados com sucesso.
✔️ Tabela dtiss2.RO_202311 feito merge e procedimentos separados com sucesso.
✔️ Tabela dtiss2.RO_202312 feito merge e procedimentos separados com sucesso.
✔️ Tabela dtiss2.RO_202401 feito merge e procedimentos separados

In [15]:
######CRIANDO AS TABELAS FINAIS POR UF

# Parâmetros
m = uf.shape[0]

for j in range(0,1):  # uf
        uf_atual = uf.iloc[j, 0]
        ##juntando as tabelas com procedimentos selecionados
s1 = f'CREATE TABLE dtiss2.{uf_atual}_amb_1 ('
s2 = '''
	id_evento_atencao_saude int8 NULL,
	id_plano int8 NULL,
	idade_beneficiario text NULL,
	faixa_etaria text NULL,
	sexo text NULL,
	cd_municipio_beneficiario float8 NULL,
	porte text NULL,
	cd_modalidade int8 NULL,
	nm_modalidade text NULL,
	cd_municipio_prestador int8 NULL,
	uf_prestador text NULL,
	dt_realizacao text NULL,
	cd_carater_atendimento float8 NULL,
	tipo_atendimento float8 NULL,
	cd_motivo_saida float8 NULL,
	cbo float8 NULL,
	tipo_consulta float8 NULL,
	ind_acidente_doenca float8 NULL,
	lg_valor_preestabelecido int8 NULL,
	ano_realizacao text NULL,
	cd_procedimento text NULL,
	cd_tabela_referencia int8 NULL,
	qt_procedimento numeric NULL,
	vl_procedimento numeric NULL,
	vl_procedimento_fornecedor numeric NULL,
	ind_pacote int8 NULL,
	ind_tabela_propria int8 NULL,
	lg_outlier text NULL
);
'''
sql = s1 + s2
executar_sql (sql)  
    ##juntando as tabelas com todos os dados
s1 = f'CREATE TABLE dtiss2.{uf_atual}_amb ('
s2 = '''
	id_evento_atencao_saude int8 NULL,
	id_plano int8 NULL,
	idade_beneficiario text NULL,
	faixa_etaria text NULL,
	sexo text NULL,
	cd_municipio_beneficiario float8 NULL,
	porte text NULL,
	cd_modalidade int8 NULL,
	nm_modalidade text NULL,
	cd_municipio_prestador int8 NULL,
	uf_prestador text NULL,
	dt_realizacao text NULL,
	cd_carater_atendimento float8 NULL,
	tipo_atendimento float8 NULL,
	cd_motivo_saida float8 NULL,
	cbo float8 NULL,
	tipo_consulta float8 NULL,
	ind_acidente_doenca float8 NULL,
	lg_valor_preestabelecido int8 NULL,
	ano_realizacao text NULL,
	cd_procedimento text NULL,
	cd_tabela_referencia int8 NULL,
	qt_procedimento numeric NULL,
	vl_procedimento numeric NULL,
	vl_procedimento_fornecedor numeric NULL,
	ind_pacote int8 NULL,
	ind_tabela_propria int8 NULL,
	lg_outlier text NULL
);'''
sql = s1 + s2
executar_sql (sql)      
print(f'✔️ Tabela dtiss2.{uf_atual} CRIADA com sucesso.')

✔️ Tabela dtiss2.RO CRIADA com sucesso.


In [ ]:
#####juntando as tabelas mensais por uf
# Parâmetros
n = anomes.shape[0]
m = uf.shape[0]

for i in range(0,n):      # anomes
    for j in range(0,1):  # uf
        uf_atual = uf.iloc[j, 0]
        anomes_atual = anomes.iloc[i, 0]
        #juntando as tabelas em uma
        s1 = f'INSERT INTO dtiss2.{uf_atual}_amb_1 '
        s2 = f'SELECT *FROM dtiss2.{uf_atual}_amb_1_{anomes_atual} '
        sql = s1 + s2
        executar_sql (sql)
        ##juntando as tabelas com todos os dados
        #juntando as tabelas em uma
        s1 = f'INSERT INTO dtiss2.{uf_atual}_amb '
        s2 = f'SELECT *FROM dtiss2.{uf_atual}_amb_{anomes_atual} '
        sql = s1 + s2
        executar_sql (sql)
        print(f'✔️ Tabela dtiss2.{uf_atual}_{anomes_atual} JUNTA com sucesso.')



✔️ Tabela dtiss2.RO_202301 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202302 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202303 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202304 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202305 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202306 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202307 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202308 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202309 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202310 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202311 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202312 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202401 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202402 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202403 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202404 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202405 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202406 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202407 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202408 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202409 JUNTA com sucesso.
✔️ Tabela dtiss2.RO_202410 JUNTA c

In [ ]:
#####CRIANDO AS TABELAS BR

##juntando as tabelas com procedimentos selecionados
s1 = f'CREATE TABLE dtiss2.br_amb_1 ('
s2 = '''
	id_evento_atencao_saude int8 NULL,
	id_plano int8 NULL,
	idade_beneficiario text NULL,
	faixa_etaria text NULL,
	sexo text NULL,
	cd_municipio_beneficiario float8 NULL,
	porte text NULL,
	cd_modalidade int8 NULL,
	nm_modalidade text NULL,
	cd_municipio_prestador int8 NULL,
	uf_prestador text NULL,
	dt_realizacao text NULL,
	cd_carater_atendimento float8 NULL,
	tipo_atendimento float8 NULL,
	cd_motivo_saida float8 NULL,
	cbo float8 NULL,
	tipo_consulta float8 NULL,
	ind_acidente_doenca float8 NULL,
	lg_valor_preestabelecido int8 NULL,
	ano_realizacao text NULL,
	cd_procedimento text NULL,
	cd_tabela_referencia int8 NULL,
	qt_procedimento numeric NULL,
	vl_procedimento numeric NULL,
	vl_procedimento_fornecedor numeric NULL,
	ind_pacote int8 NULL,
	ind_tabela_propria int8 NULL,
	lg_outlier text NULL
);
'''
sql = s1 + s2
executar_sql (sql)  


##juntando as tabelas com procedimentos selecionados
s1 = f'CREATE TABLE dtiss2.br_amb ('
s2 = '''
	id_evento_atencao_saude int8 NULL,
	id_plano int8 NULL,
	idade_beneficiario text NULL,
	faixa_etaria text NULL,
	sexo text NULL,
	cd_municipio_beneficiario float8 NULL,
	porte text NULL,
	cd_modalidade int8 NULL,
	nm_modalidade text NULL,
	cd_municipio_prestador int8 NULL,
	uf_prestador text NULL,
	dt_realizacao text NULL,
	cd_carater_atendimento float8 NULL,
	tipo_atendimento float8 NULL,
	cd_motivo_saida float8 NULL,
	cbo float8 NULL,
	tipo_consulta float8 NULL,
	ind_acidente_doenca float8 NULL,
	lg_valor_preestabelecido int8 NULL,
	ano_realizacao text NULL,
	cd_procedimento text NULL,
	cd_tabela_referencia int8 NULL,
	qt_procedimento numeric NULL,
	vl_procedimento numeric NULL,
	vl_procedimento_fornecedor numeric NULL,
	ind_pacote int8 NULL,
	ind_tabela_propria int8 NULL,
	lg_outlier text NULL
);
'''
sql = s1 + s2
executar_sql (sql)  

*2.5 tabela dtiss AMBULATORIAL BR*

In [ ]:
#####ALIMENTANDO A TABELA MAE COM AS TABELAS FILHAS
m = uf.shape[0]

for j in range(0,1):  # uf  
uf_atual = uf.iloc[j, 0]
###base com os procedimentos selecionados   
sintaxe = f'ALTER TABLE dtiss2.{uf_atual}_amb_1 INHERIT dtiss2.br_amb_1 '
sql = ( sintaxe )
executar_sql (sql)
###base sem os procedimentos selecionados   
sintaxe = f'ALTER TABLE dtiss2.{uf_atual}_amb INHERIT dtiss2.br_amb '
sql = ( sintaxe )
executar_sql (sql)
print(f'✔️ Tabela dtiss2.{uf_atual} JUNTA na tabela BR DTISS com sucesso.')

*2.6 FREQUENCIA*

In [ ]:
#######criando a tabela dtiss procedimentos 2022
sintaxe = '''

create table dtiss2.br_amb_1_proced as
select cd_procedimento,ano_realizacao , 
sum (qt_procedimento) as N_procedimento, sum (vl_procedimento) as custo_procedimento 
from dtiss2.br_amb_1
group by cd_procedimento,ano_realizacao;

'''
sql = ( sintaxe )
executar_sql (sql)
print(["✔️---------TABELA DTISS AGREGADA CRIADA com sucesso ---------"] ) 

In [ ]:
#######criando a tabela dtiss procedimentos POR MODALIDADE DE OPERADORA 2022
sintaxe = '''

create table dtiss2.br_amb_1__MO as
select cd_procedimento,ano_realizacao , CD_MODALIDADE, NM_MODALIDADE,
sum (qt_procedimento) as N_procedimento, sum (vl_procedimento) as custo_procedimento 
from dtiss2.br_amb_1
group by cd_procedimento,ano_realizacao,CD_MODALIDADE, NM_MODALIDADE;

'''
sql = ( sintaxe )
executar_sql (sql)
print(["✔️---------TABELA DTISS AGREGADA CRIADA com sucesso ---------"] ) 